In [24]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from PINN.utilities import V_BS

In [ ]:
# parameters
K = 4
sigma = 0.3
r = 0.03
T = 1
S_inf = 3 * K  # truncted domain (0, inf]

np.random.seed(42)
torch.manual_seed(42);

In [ ]:
#### European put option
N_i = 160
N_b = 40
N_pde = 2500
N_data = 0

# make data
# random sample S in (0, S_inf), tau in (0, 1)
# initial condition V(S, 0) = max(K - S, 0)
S_ib = torch.rand(N_i, 1)*S_inf
tau_ib = torch.zeros(N_i, 1)
V_ib = torch.max(K - S_ib, torch.zeros_like(S_ib))
# boundary condition V(0, tau) = K*exp(-r*tau), V(S_inf, tau) = 0
S_ib = torch.cat([S_ib, torch.zeros(N_b, 1), torch.full((N_b, 1), S_inf)], dim=0)
tau_ib = torch.cat([tau_ib, torch.rand(2*N_b, 1)*T], dim=0)
V_ib = torch.cat([V_ib, K*torch.exp(-r*tau_ib[N_i:N_i+N_b]), torch.zeros((N_b, 1))], dim=0)
# PDE points
S_pde = torch.rand(N_pde, 1)*S_inf
tau_pde = torch.rand(N_pde, 1)

# require_grad = True
S_ib.requires_grad = True
tau_ib.requires_grad = True
V_ib.requires_grad = True
S_pde.requires_grad = True
tau_pde.requires_grad = True


# # save data
# torch.save({'S': S_ib, 'tau': tau_ib, 'V': V_ib}, './data/european_put_ib.pt')
# torch.save({'S': S_pde, 'tau': tau_pde}, './data/european_put_pde.pt')

In [20]:
#### European call option
N_i = 160
N_b = 40
N_pde = 5000
N_data = 40
# make data
# random sample S in (0, S_inf), tau in (0, 1)
# initial condition V(S, 0) = max(S - K, 0)
S_ib = torch.rand(N_i, 1)*S_inf
tau_ib = torch.zeros(N_i, 1)
V_ib = torch.max(S_ib - K, torch.zeros_like(S_ib))
# boundary condition V(0, tau) = 0, V(S_inf, tau) = S_inf - K*exp(-r*tau)
S_ib = torch.cat([S_ib, torch.zeros(N_b, 1), torch.full((N_b, 1), S_inf)], dim=0)
tau_ib = torch.cat([tau_ib, torch.rand(2*N_b, 1)], dim=0)
V_ib = torch.cat([V_ib, torch.zeros((N_b, 1)), torch.full((N_b, 1), S_inf)-K*torch.exp(-r*tau_ib[N_i:N_i+N_b])], dim=0)
# PDE points
S_pde = torch.rand(N_pde, 1)*S_inf
tau_pde = torch.rand(N_pde, 1)
#  data points
if N_data > 0:
    S_data = torch.rand(N_data, 1)*S_inf
    tau_data = torch.rand(N_data, 1)
    V_data = V_BS(S_data, tau_data, K, r, sigma, 'call')
    S_data.requires_grad = True
    tau_data.requires_grad = True
    V_data.requires_grad = True
else:
    S_data = None
    tau_data = None
    V_data = None

# require_grad = True
S_ib.requires_grad = True
tau_ib.requires_grad = True
V_ib.requires_grad = True
S_pde.requires_grad = True
tau_pde.requires_grad = True

# save data
torch.save({'S': S_ib, 'tau': tau_ib, 'V': V_ib}, './data/european_call_ib.pt')
torch.save({'S': S_pde, 'tau': tau_pde}, './data/european_call_pde.pt')
if N_data > 0:
    torch.save({'S': S_data, 'tau': tau_data, 'V': V_data}, './data/european_call_data.pt')